In [1]:
import gym
import copy
import numpy as np
import time as time
import torch
import torch.nn as nn
import gym_flappyBird
import genetics as gen
from tensorboardX import SummaryWriter
import matplotlib.pyplot as plt
import math
import pygame
from pygame.locals import *

pygame 2.0.2 (SDL 2.0.16, Python 3.8.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


# Maschine

## Feature extrahieren

<img src="sprites/ingame.jpg" align="left" >

In [2]:
def generateFeatures(state):
    bird = state['bird']
    #print(type(bird))
    posY = bird.Y
    speedY = bird.speedY
    pipes = state['pipes']
    #print(pipes[0].pos)
    return posY, speedY, pipes[0].pos,pipes[0].height,pipes[0].gap

## Reward  berechnen

<img src="sprites/ingame_2.jpg" align="left" >

In [3]:
def computeReward(state_old, state_new):
    return 0

# Aktion festlegen

In [4]:
def birdAction(decission, bird):
        bird.force = [0.0, 50*decission[0]]

# Mutationsparameter

<img src="sprites/ingame_3.jpg" align="left" >

In [5]:
NOISE_STD = 0.5
POPULATION_SIZE = 250
PARENTS_COUNT = 15

# Neunales Netz 

## Layer
nn.Linear(X, Y), Input dim X, Output dim Y

## Aktivierungsfunktionen
nn.ReLU()


nn.Sigmoid()


nn.Softmax(dim=1))



## Topologie
nn.Sequential(Layer, Aktivierungsfunktion, Layer, Aktivierungsfunktion...., Layer,Softmax)
###### Beispiel
nn.Sequential(nn.Linear(5, 5),nn.Softmax(dim=1))

In [6]:
net = nn.Sequential(nn.Linear(5, 5),nn.Sigmoid(),nn.Linear(5, 5),nn.Sigmoid(), nn.Linear(5, 1))

In [7]:
#net = nn.Sequential(nn.Linear(5, 5),nn.ReLU(),nn.Linear(5, 2),nn.Softmax(dim=1))

In [8]:
for name, param in net.named_parameters():
    print(name, param)

0.weight Parameter containing:
tensor([[-0.3992, -0.0735,  0.0805, -0.3926, -0.0434],
        [ 0.1759, -0.0456, -0.2572,  0.2926, -0.0614],
        [-0.0841,  0.2410, -0.1037, -0.0786,  0.0297],
        [-0.1184,  0.2544,  0.1787, -0.1755,  0.2791],
        [-0.2037, -0.4385, -0.2987, -0.0054, -0.4157]], requires_grad=True)
0.bias Parameter containing:
tensor([ 0.4446, -0.1117, -0.1708, -0.3057,  0.3649], requires_grad=True)
2.weight Parameter containing:
tensor([[ 0.3430, -0.3011, -0.2682, -0.0816,  0.0942],
        [ 0.2432,  0.0040,  0.3735, -0.0808,  0.1369],
        [-0.1409, -0.0059,  0.2481,  0.0280, -0.2164],
        [-0.0387, -0.3562, -0.3306, -0.3480, -0.3215],
        [ 0.1270, -0.3482, -0.1871,  0.3036,  0.2014]], requires_grad=True)
2.bias Parameter containing:
tensor([-0.3598,  0.1205, -0.3648, -0.0759, -0.1977], requires_grad=True)
4.weight Parameter containing:
tensor([[-0.0448,  0.1087,  0.1858,  0.4039, -0.4243]], requires_grad=True)
4.bias Parameter containing:
tens

## [ Abstand zwischen,  Höhe der Pipes, Grösse der Lücke

In [9]:
Interval_distance = [250, 350]
Interval_height = [100,300]
Interval_gap = [120,130]

## Max Score

In [10]:
Score_Max = 4000

In [11]:
fittestBirds = []

env = gym.make("scienceCampBird-v1")
env.setPipeIntervals([Interval_distance, Interval_height,Interval_gap])
population = gen.Population(POPULATION_SIZE, 5, 2, computeReward, net)
env.setAction(birdAction)
  #  print(len(population.nets))
population.evaluate_on_env(env,generateFeatures, Score_Max)
ecount = 0  
while True:
    population = gen.mutate_population(population, PARENTS_COUNT, NOISE_STD)
    population.evaluate_on_env(env, generateFeatures, Score_Max)
    fittestBirds.append(population.population[0])
    ecount +=1
    if(ecount % 20 == 0):
        net = population.population[0][1]
        score_e = population.population[0][0]
        score_p = env.playWithNet(net, generateFeatures, Score_Max, computeReward, ecount)
        print("----------------------------------------------------------------------------")
        print("----------------------------------------------------------------------------")
        print('Population: ', ecount)
        print("____________________________________________________________________________")
        print('Score Training: ', score_e, ' Score Spiel: ', score_p)
        print("____________________________________________________________________________")
        for name, param in net.named_parameters():
            print(name, param)

----------------------------------------------------------------------------
----------------------------------------------------------------------------
Population:  20
____________________________________________________________________________
Score Training:  0.0  Score Spiel:  0.0
____________________________________________________________________________
0.weight Parameter containing:
tensor([[-0.3992, -0.0735,  0.0805, -0.3926, -0.0434],
        [ 0.1759, -0.0456, -0.2572,  0.2926, -0.0614],
        [-0.0841,  0.2410, -0.1037, -0.0786,  0.0297],
        [-0.1184,  0.2544,  0.1787, -0.1755,  0.2791],
        [-0.2037, -0.4385, -0.2987, -0.0054, -0.4157]], requires_grad=True)
0.bias Parameter containing:
tensor([ 0.4446, -0.1117, -0.1708, -0.3057,  0.3649], requires_grad=True)
2.weight Parameter containing:
tensor([[ 0.3430, -0.3011, -0.2682, -0.0816,  0.0942],
        [ 0.2432,  0.0040,  0.3735, -0.0808,  0.1369],
        [-0.1409, -0.0059,  0.2481,  0.0280, -0.2164],
        [-

KeyboardInterrupt: 

# Plott

In [ ]:
scores = [score[0] for score in fittestBirds]
fig, ax = plt.subplots()
ax.plot(scores)

ax.set(xlabel='Pupulation', ylabel='Score',
       title=' Scores der besten Vogels der Population')
ax.grid()

fig.savefig("test.png")
plt.show()

In [ ]:
i = 0
for bird in fittestBirds:
        i +=1
        net = bird[1]
        score_p = bird[0]
        print("----------------------------------------------------------------------------")
        print("----------------------------------------------------------------------------")
        print('Population: ', i)
        print("____________________________________________________________________________")
        print('Score: ', score_p)
        print("____________________________________________________________________________")
        for name, param in net.named_parameters():
            print(name, param)